In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import calendar
from time import time
import re 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import log_loss, accuracy_score, classification_report
%matplotlib inline

In [2]:
data_train = pd.read_csv("train.csv").replace("MENS WEAR","MENSWEAR")
data_train_orig = pd.read_csv("train.csv").replace("MENS WEAR","MENSWEAR")

In [3]:
dept_list = sorted(list(data_train.DepartmentDescription.dropna().unique()))

In [4]:
weekdays = list(calendar.day_name)
dept_list_sum = dict.fromkeys(dept_list, np.sum)
weekday_dict = dict.fromkeys(weekdays, np.max)
feature_dict = {"TripType": np.max, 'NumItems': np.sum, 'Return': np.max}
feature_dict = {**feature_dict, **weekday_dict, **dept_list_sum}

In [5]:
def transform_data(data):
    dummies = pd.get_dummies(data.Weekday)
    data[dummies.columns] = dummies
    
    dummies = pd.get_dummies(data.DepartmentDescription)
    dummies = dummies.apply(lambda x: x*data["ScanCount"])
    data[dummies.columns] = dummies 

    data.loc[data.ScanCount < 0, 'Return'] = 1
    data.loc[data.Return != 1, 'Return'] = 0
    
    data = data.rename(columns={"ScanCount":"NumItems"})
    
    grouped = data.groupby("VisitNumber")
    grouped = grouped.aggregate(feature_dict)
    data = grouped[["TripType", "NumItems", "Return"] + weekdays + dept_list]

    return data

In [6]:
data_new = transform_data(data_train)

In [7]:
def add_category_counts(data):
    alist = []
    for array in np.asarray(data.loc[:, dept_list[0]:]):
        count = 0
        count = sum(x > 0 for x in array)
        alist.append(count)
    cat_counts = pd.DataFrame(alist)
    cat_counts = cat_counts.rename(columns={0:"CategoryCount"})
    cat_counts = cat_counts.set_index(data.index)
    data.insert(3, 'CategoryCounts', cat_counts)
    return data

In [8]:
data_new_cat = add_category_counts(data_new)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD

Using TensorFlow backend.


In [10]:
X = data_new_cat.drop('TripType', axis=1)

trip_types = sorted(data_new_cat.TripType.unique())
trip_types_map = dict(zip(trip_types, np.arange(0, len(trip_types))))
y = data_new_cat.TripType.map(trip_types_map)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [32]:
np.unique(y_train.to_numpy())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37], dtype=int64)

In [33]:
np.unique(y_test.to_numpy())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37], dtype=int64)

In [11]:
np.random.seed(42)

In [12]:
def create_model_1hl(nodes, dropout):
    model = Sequential()
    model.add(Dense(output_dim=nodes, activation="relu", input_dim=len(X.columns)))
    model.add(Dropout(dropout))
            
    model.add(Dense(output_dim=len(trip_types), activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return model

In [13]:
model_1hl = KerasClassifier(build_fn=create_model_1hl, epochs=50, verbose=2)

In [14]:
nodes_1hl = np.arange(50, 100, 10)

dropout1 = np.arange(0, 0.5, 0.1)
dropout2 = np.arange(0, 0.5, 0.1)
dropout3 = np.arange(0, 0.5, 0.1)
batch_size = [500, 1000, 2000, 5000]

In [15]:
params1 = dict(nodes=nodes_1hl, dropout=dropout1, batch_size=batch_size)

In [16]:
params1

{'nodes': array([50, 60, 70, 80, 90]),
 'dropout': array([0. , 0.1, 0.2, 0.3, 0.4]),
 'batch_size': [500, 1000, 2000, 5000]}

In [17]:
scoring = ['neg_log_loss', 'accuracy']

In [34]:
from sklearn.model_selection import StratifiedKFold

In [36]:
RS1 = RandomizedSearchCV(estimator=model_1hl, param_distributions=params1, cv=StratifiedKFold(n_splits=4), n_iter = 3, 
                         scoring=scoring, refit='accuracy', random_state=42, n_jobs=1, verbose=2)

In [37]:
RS1

RandomizedSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
                   error_score='raise-deprecating',
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001F01EB471D0>,
                   iid='warn', n_iter=3, n_jobs=1,
                   param_distributions={'batch_size': [500, 1000, 2000, 5000],
                                        'dropout': array([0. , 0.1, 0.2, 0.3, 0.4]),
                                        'nodes': array([50, 60, 70, 80, 90])},
                   pre_dispatch='2*n_jobs', random_state=42, refit='accuracy',
                   return_train_score=False,
                   scoring=['neg_log_loss', 'accuracy'], verbose=2)

In [38]:
start = time()
RS1.fit(X, y)
end = time()
end - start

Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=80)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


[CV] nodes=80, dropout=0.1, batch_size=5000 ..........................
Epoch 1/50
 - 1s - loss: 3.7065 - acc: 0.0836
Epoch 2/50
 - 1s - loss: 3.0598 - acc: 0.2508
Epoch 3/50
 - 1s - loss: 2.7543 - acc: 0.3777
Epoch 4/50
 - 1s - loss: 2.4995 - acc: 0.4548
Epoch 5/50
 - 1s - loss: 2.2756 - acc: 0.4977
Epoch 6/50
 - 1s - loss: 2.0822 - acc: 0.5298
Epoch 7/50
 - 1s - loss: 1.9187 - acc: 0.5504
Epoch 8/50
 - 1s - loss: 1.7730 - acc: 0.5729
Epoch 9/50
 - 1s - loss: 1.6567 - acc: 0.5911
Epoch 10/50
 - 1s - loss: 1.5606 - acc: 0.6040
Epoch 11/50
 - 1s - loss: 1.4811 - acc: 0.6145
Epoch 12/50
 - 1s - loss: 1.4141 - acc: 0.6232
Epoch 13/50
 - 1s - loss: 1.3576 - acc: 0.6327
Epoch 14/50
 - 1s - loss: 1.3084 - acc: 0.6396
Epoch 15/50
 - 1s - loss: 1.2705 - acc: 0.6444
Epoch 16/50
 - 1s - loss: 1.2362 - acc: 0.6485
Epoch 17/50
 - 1s - loss: 1.2088 - acc: 0.6534
Epoch 18/50
 - 1s - loss: 1.1826 - acc: 0.6570
Epoch 19/50
 - 1s - loss: 1.1607 - acc: 0.6604
Epoch 20/50
 - 1s - loss: 1.1409 - acc: 0.662

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.2s remaining:    0.0s
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=80)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/50
 - 1s - loss: 3.5220 - acc: 0.1198
Epoch 2/50
 - 1s - loss: 3.0620 - acc: 0.2780
Epoch 3/50
 - 1s - loss: 2.7704 - acc: 0.4033
Epoch 4/50
 - 1s - loss: 2.5138 - acc: 0.4753
Epoch 5/50
 - 1s - loss: 2.2889 - acc: 0.5170
Epoch 6/50
 - 1s - loss: 2.0876 - acc: 0.5464
Epoch 7/50
 - 1s - loss: 1.9188 - acc: 0.5632
Epoch 8/50
 - 1s - loss: 1.7735 - acc: 0.5794
Epoch 9/50
 - 1s - loss: 1.6546 - acc: 0.5918
Epoch 10/50
 - 1s - loss: 1.5556 - acc: 0.6043
Epoch 11/50
 - 1s - loss: 1.4743 - acc: 0.6159
Epoch 12/50
 - 1s - loss: 1.4070 - acc: 0.6253
Epoch 13/50
 - 1s - loss: 1.3491 - acc: 0.6317
Epoch 14/50
 - 1s - loss: 1.3056 - acc: 0.6375
Epoch 15/50
 - 1s - loss: 1.2656 - acc: 0.6435
Epoch 16/50
 - 1s - loss: 1.2335 - acc: 0.6479
Epoch 17/50
 - 1s - loss: 1.2030 - acc: 0.6524
Epoch 18/50
 - 1s - loss: 1.1771 - acc: 0.6578
Epoch 19/50
 - 1s - loss: 1.1564 - acc: 0.6589
Epoch 20/50
 - 1s - loss: 1.1378 - acc: 0.6620
Epoch 21/50
 - 1s - loss: 1.1225 - acc: 0.6625
Epoch 22/50
 - 1s - lo

...

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=50)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/50
 - 1s - loss: 3.7201 - acc: 0.1100
Epoch 2/50
 - 1s - loss: 3.0253 - acc: 0.2548
Epoch 3/50
 - 1s - loss: 2.6179 - acc: 0.3537
Epoch 4/50
 - 1s - loss: 2.2979 - acc: 0.4310
Epoch 5/50
 - 1s - loss: 2.0512 - acc: 0.4847
Epoch 6/50
 - 1s - loss: 1.8659 - acc: 0.5211
Epoch 7/50
 - 1s - loss: 1.7273 - acc: 0.5504
Epoch 8/50
 - 1s - loss: 1.6185 - acc: 0.5702
Epoch 9/50
 - 1s - loss: 1.5377 - acc: 0.5846
Epoch 10/50
 - 1s - loss: 1.4758 - acc: 0.5976
Epoch 11/50
 - 1s - loss: 1.4262 - acc: 0.6052
Epoch 12/50
 - 1s - loss: 1.3867 - acc: 0.6111
Epoch 13/50
 - 1s - loss: 1.3496 - acc: 0.6177
Epoch 14/50
 - 1s - loss: 1.3202 - acc: 0.6235
Epoch 15/50
 - 1s - loss: 1.2989 - acc: 0.6265
Epoch 16/50
 - 1s - loss: 1.2749 - acc: 0.6316
Epoch 17/50
 - 1s - loss: 1.2605 - acc: 0.6325
Epoch 18/50
 - 1s - loss: 1.2481 - acc: 0.6354
Epoch 19/50
 - 1s - loss: 1.2340 - acc: 0.6382
Epoch 20/50
 - 1s - loss: 1.2221 - acc: 0.6390
Epoch 21/50
 - 1s - loss: 1.2138 - acc: 0.6400
Epoch 22/50
 - 1s - lo

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  6.9min finished
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=80)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/50
 - 2s - loss: 3.1363 - acc: 0.2386
Epoch 2/50
 - 1s - loss: 2.2171 - acc: 0.5203
Epoch 3/50
 - 1s - loss: 1.7054 - acc: 0.5929
Epoch 4/50
 - 1s - loss: 1.4199 - acc: 0.6297
Epoch 5/50
 - 1s - loss: 1.2596 - acc: 0.6534
Epoch 6/50
 - 1s - loss: 1.1667 - acc: 0.6644
Epoch 7/50
 - 1s - loss: 1.1114 - acc: 0.6716
Epoch 8/50
 - 1s - loss: 1.0750 - acc: 0.6763
Epoch 9/50
 - 1s - loss: 1.0486 - acc: 0.6793
Epoch 10/50
 - 1s - loss: 1.0277 - acc: 0.6822
Epoch 11/50
 - 1s - loss: 1.0102 - acc: 0.6839
Epoch 12/50
 - 1s - loss: 0.9943 - acc: 0.6864
Epoch 13/50
 - 1s - loss: 0.9805 - acc: 0.6882
Epoch 14/50
 - 1s - loss: 0.9681 - acc: 0.6893
Epoch 15/50
 - 1s - loss: 0.9568 - acc: 0.6927
Epoch 16/50
 - 1s - loss: 0.9472 - acc: 0.6939
Epoch 17/50
 - 1s - loss: 0.9383 - acc: 0.6953
Epoch 18/50
 - 1s - loss: 0.9308 - acc: 0.6964
Epoch 19/50
 - 1s - loss: 0.9236 - acc: 0.6971
Epoch 20/50
 - 1s - loss: 0.9166 - acc: 0.6983
Epoch 21/50
 - 1s - loss: 0.9105 - acc: 0.6998
Epoch 22/50
 - 1s - lo

457.38472032546997

In [39]:
pd.DataFrame(RS1.cv_results_).columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_nodes', 'param_dropout', 'param_batch_size', 'params',
       'split0_test_neg_log_loss', 'split1_test_neg_log_loss',
       'split2_test_neg_log_loss', 'split3_test_neg_log_loss',
       'mean_test_neg_log_loss', 'std_test_neg_log_loss',
       'rank_test_neg_log_loss', 'split0_test_accuracy',
       'split1_test_accuracy', 'split2_test_accuracy', 'split3_test_accuracy',
       'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy'],
      dtype='object')

In [40]:
RS1.best_params_

{'nodes': 80, 'dropout': 0.0, 'batch_size': 2000}

In [41]:
RS1.best_score_

0.6991659175951669

In [47]:
cols = ['rank_test_neg_log_loss', 'mean_test_neg_log_loss', 'rank_test_accuracy', 'mean_test_accuracy', 
        'param_nodes', 'param_dropout', 'param_batch_size']

In [48]:
df = pd.DataFrame(RS1.cv_results_)
df[cols]

,rank_test_neg_log_loss,mean_test_neg_log_loss,rank_test_accuracy,mean_test_accuracy,param_nodes,param_dropout,param_batch_size
0,2,-0.938457,2,0.692403,80,0.1,5000
1,1,-0.908023,1,0.699166,80,0,2000
2,3,-0.954272,3,0.691818,50,0.4,2000


In [49]:
RS = RandomizedSearchCV(estimator=model_1hl, param_distributions=params1, cv=StratifiedKFold(n_splits=4), n_iter = 3, 
                         scoring=scoring, refit='accuracy', random_state=42, n_jobs=-1, verbose=2)

In [50]:
start = time()
RS.fit(X, y)
end = time()
end - start

Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  5.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  5.8min finished
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=80)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/50
 - 2s - loss: 2.9088 - acc: 0.3046
Epoch 2/50
 - 1s - loss: 2.1415 - acc: 0.5170
Epoch 3/50
 - 1s - loss: 1.6756 - acc: 0.5834
Epoch 4/50
 - 1s - loss: 1.4077 - acc: 0.6296
Epoch 5/50
 - 1s - loss: 1.2570 - acc: 0.6528
Epoch 6/50
 - 1s - loss: 1.1689 - acc: 0.6633
Epoch 7/50
 - 1s - loss: 1.1140 - acc: 0.6704
Epoch 8/50
 - 1s - loss: 1.0766 - acc: 0.6755
Epoch 9/50
 - 1s - loss: 1.0492 - acc: 0.6787
Epoch 10/50
 - 1s - loss: 1.0280 - acc: 0.6807
Epoch 11/50
 - 1s - loss: 1.0102 - acc: 0.6841
Epoch 12/50
 - 1s - loss: 0.9953 - acc: 0.6843
Epoch 13/50
 - 1s - loss: 0.9817 - acc: 0.6873
Epoch 14/50
 - 1s - loss: 0.9696 - acc: 0.6883
Epoch 15/50
 - 1s - loss: 0.9590 - acc: 0.6903
Epoch 16/50
 - 1s - loss: 0.9490 - acc: 0.6918
Epoch 17/50
 - 1s - loss: 0.9409 - acc: 0.6939
Epoch 18/50
 - 1s - loss: 0.9324 - acc: 0.6951
Epoch 19/50
 - 1s - loss: 0.9253 - acc: 0.6964
Epoch 20/50
 - 1s - loss: 0.9190 - acc: 0.6966
Epoch 21/50
 - 1s - loss: 0.9129 - acc: 0.6975
Epoch 22/50
 - 1s - lo

388.86159896850586

In [51]:
dff = pd.DataFrame(RS.cv_results_)
dff[cols]

,rank_test_neg_log_loss,mean_test_neg_log_loss,rank_test_accuracy,mean_test_accuracy,param_nodes,param_dropout,param_batch_size
0,2,-0.937103,2,0.693334,80,0.1,5000
1,1,-0.908798,1,0.696992,80,0,2000
2,3,-0.954938,3,0.690041,50,0.4,2000
